# Import of needed packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')
import ipywidgets as widgets
%matplotlib inline

# Introduction
In this project we fetch data from Statstics Denmark in order to explore the share of highly educated, the share of immigrants, the share of people on social benefits (i.e. 'kontanthjælp') and the average income in the Danish municipalities. The datasets used are:
* INDKP106
* FOLK1C
* KUY038
* HFUDD10

The structure of the notebook is as follows:
* Fetch Data - the section will download the data described above using Statistics Denmark's API and clean it
* Merge Data - the section merges our cleaned data into dataframa containing data for all Danish municipalities across years
* Data Visualization and Analysis - the section will visualize data and explore correlations

The goal of the project is to: 
* visually explore the relationship between the share of highly educated and average income in all municipalities
* visually explore the relationship between the share of immigrants and the share of people on social benefits in all municipalities

* compare the demeaned devolopment of the share of people on social benefits in the largest municipalities

# Motivation
Before the Covid-19 pandemic the hot topic at Chrstiansborg was “Udligningsreformen” which is a reform set to equalize municipality budgets. Some municipalities like Rudersdal is characterized by a high concentration of wealthy people who both pays high amounts of taxes and also require low levels of social benefits. Other municipalities are characterized by a high concentration of low-income people which leads to relatively low amount of taxes and relatively high amount of social benefits. Hence, the demographic of municipalities has a great impact on municipalities budgets. “Udligningsreformen” is set to equalize these inequalities.
We seek to analyze different factors of inequality in municipalities in Denmark in the time period 2006-2018.

# Fetch Data
## Educational Data
### Ultimately, we want to look at the share of people who have a bachelor's degree or higher across municipalities and time. Thus, we want to look at the table HFUDD10.

In [2]:
dst.get_data(table_id = 'HFUDD10')

#Now, we take a look at the variables
hfudd_var = dst.get_variables(table_id='HFUDD10')
hfudd_var

#Next, look at the dictionary under values, to see the categories in order to specify what variables we need
udd = hfudd_var.loc[2,'values']
udd

#First, we create a list of the main categories i.e. education level higher or equal to bachelor
# Next we pick them from the HFUDD10 table and put into the dataframe hfudd10
hfudd_list = ['TOT','H60','H70','H80']
hfudd10 = dst.get_data(table_id = 'HFUDD10', variables={'BOPOMR':['*'], 'HFUDD':hfudd_list, 'TID':['*']})
hfudd10.tail()

#Drop unnecessary columns
hfudd10.drop(['ALDER','HERKOMST','KØN'], axis=1, inplace=True)
    
#Now we select all of the higher education categories and sum them across each municipality and year.
group = hfudd10.loc[(hfudd10['HFUDD'] != 'Total')].copy() #Split
group1 = group.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum() #Apply
group2 = pd.DataFrame(group1)
group2.rename(columns = {'INDHOLD':'HIGH_EDU'}, inplace=True) #Rename column

#Select the total i.e. denominator for share. Agian in each municipality and year.
total = hfudd10.loc[(hfudd10['HFUDD'] == 'Total')].copy() #Split
total1 = total.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum() #Apply
total2 = pd.DataFrame(total1)
total2.rename(columns = {'INDHOLD':'TOTAL'}, inplace=True) #Rename column

#Merge the data, reset index and set some proper variable names
merge = pd.merge(group2,total2,how='left',on=['BOPOMR','TID']) #Combine
merge['HIGH_EDU_SHARE'] = merge['HIGH_EDU'] / merge['TOTAL'] * 100 #Share of highly educated
merge.reset_index(inplace=True)
merge.rename(columns = {'BOPOMR':'MUNICIPALITY','TID':'YEAR'}, inplace=True) #Rename column
hfudd10 = merge

#Delete all rows that is not a municipality in the residence column (the 'MUNICIPALITY' column)
for val in ['Region', 'Province', 'All Denmark']: 
    I = hfudd10.MUNICIPALITY.str.contains(val)
    hfudd10 = hfudd10.loc[I == False]

hfudd10.head()

,MUNICIPALITY,YEAR,HIGH_EDU,TOTAL,HIGH_EDU_SHARE
0,Aabenraa,2006,1322,41865,3.157769
1,Aabenraa,2007,1367,41938,3.259574
2,Aabenraa,2008,1449,42097,3.442051
3,Aabenraa,2009,1498,42255,3.545143
4,Aabenraa,2010,1515,41920,3.614027


## Income Data
### Ultimately, we want to look at the household average income across municipality and time. Thus, we fetch the table INDKP106.

In [3]:
#Dictionary of variables that we want to look at
variables = {'OMRÅDE':['*'], 'ENHED':['118'], 'KOEN':['MOK'], 'ALDER1':['00'], 'INDKINTB':['000'], 'TID':['*']}

#Call the API
indkp106 = dst.get_data(
    table_id = 'INDKP106',
    variables = variables)

#Drop redundant variables
indkp106.drop(['ENHED', 'KOEN', 'ALDER1', 'INDKINTB'], axis = 1, inplace = True)

#Rename columns
indkp106.rename(columns = {'OMRÅDE': 'MUNICIPALITY', 'ENHED': 'UNIT', 'KOEN': 'GENDER', 'INDKINTB': 'AVGINCOME', 'TID': 'YEAR'}, inplace = True)

#Delete all that is not municipalities
for val in ['Region', 'Province', 'All Denmark']: 
    I = indkp106.MUNICIPALITY.str.contains(val)
    indkp106 = indkp106.loc[I == False]

indkp106.head()

,MUNICIPALITY,YEAR,INDHOLD
0,Høje-Taastrup,2013,205502
1,Struer,2013,196670
2,Ærø,2013,180695
3,Horsens,2013,204107
4,Aarhus,2013,207081


## Immigration Data
### Ultimately, we want to look at the share of immigrants across municipality and time. We fetch the population data in which the number of immigrants is reported.

In [4]:
#Dictionary of variables that we want to look at
variables = {'OMRÅDE':['*'],'HERKOMST':["*"],"TID":["*"]}

#Call the API
folk_api = dst.get_data(table_id = 'FOLK1C', variables=variables)

#We match the observation at 1st of january with the other datasets, thus selecting the first quarter (=Q1)
folk_q1 = folk_api[folk_api["TID"].str.contains("Q1")].copy()

#Convert the time variabel to datetime format that python knows
folk_q1["TID"]=pd.to_datetime(folk_q1["TID"])

#Create year variable which matches the data in section 1.1 and 1.2, such that we can merge our datasets on years in following sections
folk_q1["YEAR"] = folk_q1["TID"].dt.year

#Drop redundant variables
folk_q1 = folk_q1.drop(["TID","KØN","IELAND","ALDER"],axis=1)

#Select all of the immigrants across each municipality and year.
group_folk = folk_q1.loc[(folk_q1['HERKOMST'] == 'Immigrants')].copy() #Split
group1_folk = group_folk.groupby(['OMRÅDE', 'YEAR'])['INDHOLD'].sum() #Count
group2_folk = pd.DataFrame(group1_folk)
group2_folk.rename(columns = {'INDHOLD':'immigrants'}, inplace=True) #Rename column

#Select the total population in each municipality and year.
total_folk = folk_q1.loc[(folk_q1['HERKOMST'] == 'Total')].copy() #Split
total1_folk = total_folk.groupby(['OMRÅDE', 'YEAR'])['INDHOLD'].sum() #Count
total2_folk = pd.DataFrame(total1_folk)
total2_folk.rename(columns = {'INDHOLD':'TOTAL'}, inplace=True) #Rename column

#Now we merge the above data, reset index and get some proper variable names. 
#We also compute the share of immigrants.
folk1c = pd.merge(group2_folk,total2_folk,how='left',on=['OMRÅDE','YEAR'])
folk1c['Immigrants_share'] = folk1c['immigrants'] / folk1c['TOTAL'] * 100
folk1c.reset_index(inplace=True)
folk1c.rename(columns = {'OMRÅDE':'MUNICIPALITY','YEAR':'YEAR'}, inplace=True) #Rename column

#Delete all rows that is not a municipality in the residence column
for val in ['Region', 'Province', 'All Denmark']: 
    I = folk1c.MUNICIPALITY.str.contains(val)
    folk1c = folk1c.loc[I == False]

folk1c.head()

,MUNICIPALITY,YEAR,immigrants,TOTAL,Immigrants_share
0,Aabenraa,2008,5127,60189,8.518168
1,Aabenraa,2009,5306,60392,8.785932
2,Aabenraa,2010,5307,59978,8.848244
3,Aabenraa,2011,5376,59795,8.990718
4,Aabenraa,2012,5478,59600,9.191275


## Data on Social Benefits
### Ultimately, we want the number of people recieving social benefits across municipality and time. Thus, we fetch KY038

In [5]:
#Dictionary of variables
variables = {'OMRÅDE':['*'], 'YDELSESTYPE':['10000'], 'HERKOMST': ['00'], 'HERKOMST1':['TOT'], 'TID':['*']} 

#Call the API
ky038 = dst.get_data(
    table_id = 'KY038',
    variables = variables)

#Drop redundant variables
ky038 = ky038.drop(['HERKOMST', 'HERKOMST1',"YDELSESTYPE"], axis = 1)

#Rename colums
ky038 = ky038.rename(columns = {'OMRÅDE': 'MUNICIPALITY','TID':'YEAR',"INDHOLD":"COUNT_KTH"})

#Delete all rows that is not a municipality in the residence column
for val in ['Region', 'Province', 'All Denmark']: 
    I = ky038.MUNICIPALITY.str.contains(val)
    ky038 = ky038.loc[I == False]

ky038.head()

,MUNICIPALITY,YEAR,COUNT_KTH
0,Helsingør,2008,1402
1,Hillerød,2008,667
2,Hørsholm,2008,164
3,Rudersdal,2008,399
4,Egedal,2008,231


# Merge
## The following section will merge the cleaned datasets

In [6]:
#Overview of dimensions
print(f'folk1c {folk1c.shape}')
print(f'hfudd10 {hfudd10.shape}')
print(f'indkp {indkp106.shape}')
print(f'ky038 {ky038.shape}')

#Find out what years we want to match when we merge
for data in [folk1c, hfudd10, indkp106, ky038]:
    count = data["YEAR"].value_counts()
    print(f'Years: {count}')

folk1c (1287, 5)
hfudd10 (1386, 5)
indkp (3136, 3)
ky038 (1188, 3)
Years: 2020    99
2019    99
2018    99
2017    99
2016    99
2015    99
2014    99
2013    99
2012    99
2011    99
2010    99
2009    99
2008    99
Name: YEAR, dtype: int64
Years: 2019    99
2018    99
2017    99
2016    99
2015    99
2014    99
2013    99
2012    99
2011    99
2010    99
2009    99
2008    99
2007    99
2006    99
Name: YEAR, dtype: int64
Years: 2017    98
2015    98
1990    98
1992    98
1994    98
1996    98
1998    98
2000    98
2002    98
2004    98
2006    98
2008    98
2010    98
2012    98
2014    98
2016    98
2018    98
1987    98
1989    98
1991    98
1993    98
1995    98
1997    98
1999    98
2001    98
2003    98
2005    98
2007    98
2009    98
2011    98
2013    98
1988    98
Name: YEAR, dtype: int64
Years: 2018    99
2017    99
2016    99
2015    99
2014    99
2013    99
2012    99
2011    99
2010    99
2009    99
2008    99
2007    99
Name: YEAR, dtype: int64


The above print statement shows:
* Immigration goes from 2008-2019
* Educational goes from 2006-2018
* Income goes from 1988-2018
* Social benefits goes from 2007-2017

In order to accomodate all years we first outer join Immigration (folk1c) and Educational (hfudd10) resulting in the years being 2006-2019.
Then, we left join the other datasets (indkp106, ky038).

In [7]:
merge_df = pd.merge(folk1c, hfudd10, on=["MUNICIPALITY","YEAR"],how="outer")
merge_df2 = pd.merge(merge_df, indkp106, on=["MUNICIPALITY","YEAR"],how="left")
merge_df3 = pd.merge(merge_df2, ky038, on=["MUNICIPALITY","YEAR"],how="left" )
merge_df3.head()

,MUNICIPALITY,YEAR,immigrants,TOTAL_x,Immigrants_share,HIGH_EDU,TOTAL_y,HIGH_EDU_SHARE,INDHOLD,COUNT_KTH
0,Aabenraa,2008,5127.0,60189.0,8.518168,1449.0,42097.0,3.442051,166290.0,1018.0
1,Aabenraa,2009,5306.0,60392.0,8.785932,1498.0,42255.0,3.545143,167106.0,1213.0
2,Aabenraa,2010,5307.0,59978.0,8.848244,1515.0,41920.0,3.614027,181280.0,1351.0
3,Aabenraa,2011,5376.0,59795.0,8.990718,1571.0,41704.0,3.767025,184117.0,1312.0
4,Aabenraa,2012,5478.0,59600.0,9.191275,1614.0,41525.0,3.886815,188090.0,1339.0


In [8]:
#Final clean
merge_df3 = merge_df3.drop(["TOTAL_y"],axis=1)
merge_df3.columns=["Municipality", "Year","Immigrants","Population","Immigrants_share","High_EDU","High_EDU_share","Avg_income","Count_KTH"]

In [9]:
#Share of population on social benefits
merge_df3["Avg_KTH"]=merge_df3["Count_KTH"]/merge_df3["Population"]*100
merge_df3.head()

,Municipality,Year,Immigrants,Population,Immigrants_share,High_EDU,High_EDU_share,Avg_income,Count_KTH,Avg_KTH
0,Aabenraa,2008,5127.0,60189.0,8.518168,1449.0,3.442051,166290.0,1018.0,1.691339
1,Aabenraa,2009,5306.0,60392.0,8.785932,1498.0,3.545143,167106.0,1213.0,2.008544
2,Aabenraa,2010,5307.0,59978.0,8.848244,1515.0,3.614027,181280.0,1351.0,2.252493
3,Aabenraa,2011,5376.0,59795.0,8.990718,1571.0,3.767025,184117.0,1312.0,2.194163
4,Aabenraa,2012,5478.0,59600.0,9.191275,1614.0,3.886815,188090.0,1339.0,2.246644


# Data Visualization and Analysis

## Education and average income

In [10]:
#First, we define a function which plots the share of highly educated agianst the average income. 
#Each dot represents a municipality.
#It is done within a function such that we can incorporate a slider and make it an interactive figure.
def edu(year):
    df = merge_df3.loc[merge_df3["Year"]==year].copy()
    plt.figure(figsize=(10,10))
    plt.scatter(df["High_EDU_share"], df["Avg_income"],cmap='viridis',c=df["High_EDU_share"]) #Colour changes with the x-axis
    plt.xlabel("Share of individuals with higher education", fontsize=16)
    plt.ylabel("Average income", fontsize=16)

#Now we set the interactive slider to change the years.
#Thus, the relationship between education and average income can be seen in a preferable year.
widgets.interact(edu,year=(2008,2018))
n_widget = widgets.IntSlider(
                value=2,
                min=2008,
                max=2018)

interactive(children=(IntSlider(value=2013, description='year', max=2018, min=2008), Output()), _dom_classes=(…

* The data suggest a positive correlation between the share of highly educated people and the average income in each municipality in each year

## Share of immigrants and share of population on social benefits

In [24]:
#Again, we define the function which creates the scatter plot
def immi(year):
    df = merge_df3.loc[merge_df3["Year"]==year].copy()
    plt.figure(figsize=(10,10))
    plt.scatter(df["Immigrants_share"], df["Avg_KTH"],cmap='viridis',c=df["Immigrants_share"]) #Colour changes with the x-axis
    plt.xlabel("Share of immigrants", fontsize=16)
    plt.ylabel("Share of individuals on public support", fontsize=16)

#Set up the widget with a slider for years
widgets.interact(immi,year=(2010,2018))
n_widget = widgets.IntSlider(
                value=2,
                min=2010,
                max=2018)

interactive(children=(IntSlider(value=2014, description='year', max=2018, min=2010), Output()), _dom_classes=(…

* The data show no clear correlation. One could argue there is a slight positive correlation. It does seem spurious, however.

## Devolopment of the demeaned share of immigrants

In [26]:
#We want to see how the share of people on social benefits 'kontanthjælp' has evolved in the municipalities. 
#By demeaning the share within each municipality we get a relative measure.

#Data selection
mean = merge_df3.loc[merge_df3["Year"] != 2019].copy() #Social benefit data is not issued for 2019

def immi_share(n_largest):
    """
    Returns: 
    Plots the n_largest municipalities and their relative demeaned share of people on social benefits
    
    Args:
    n_largest (int): Number of largest populated municipalities
    
    """
    
    #Apply-Split-Combine using the transform-method to achieve the measure described above
    mean["KTH_demeaned"] = merge_df3.groupby(['Municipality'])['Avg_KTH'].transform(lambda x: x-x.mean())
    
    #choosing n_largest municipalities based on their population
    immi_mun = mean.nlargest(n_largest, ["Year","Population"])
    
    #plotting the n_largest municipalities
    plt.figure(figsize=(10,10))
    year = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    for mun in immi_mun.Municipality:
        muni = mean.loc[mean["Municipality"]== mun]
        plt.plot(year,muni["KTH_demeaned"],label=muni["Municipality"].unique())
    plt.ylabel("Public support deviation from local mean")
    plt.xlabel("Year")
    plt.legend()
    plt.plot()
    return

#Widget that makes you choose the n_largest municipalities and plots them. 
widgets.interact(immi_share, 
    n_largest = widgets.IntSlider(description='Municipalities', min=1, max=99, step=1, value=10),
);

interactive(children=(IntSlider(value=10, description='Municipalities', max=99, min=1), Output()), _dom_classe…

* We clearly see the 'kontanthjælpsreform' from mid 2013 decreasing the number of people receiving kontanthjælp. With the jobcenters in Copenhagen doing the best job relatively.